In [56]:
# !pip install pillow numpy torch opencv-python


In [57]:
from PIL import Image
import numpy as np
import glob as glob
import cv2
import torch
from PIL import Image
from model.srcnn import SRCNN
from model.sr_resnet import SRResNet
import random

# TRAIN_BATCH_SIZE = 16
# TEST_BATCH_SIZE = 1
# SHOW_PATCHES = True
# STRIDE = 100
# SIZE = 32

In [58]:
torch.cuda.empty_cache()

In [59]:
# device = 'cuda:0'
device = 'cpu'
print(device)
model = SRCNN().to(device)
# model = SRResNet(2).to(device)
model.load_state_dict(torch.load("/media/hero/Study/User/Study/data/model_out/image_out/best_train.pth", map_location=torch.device('cpu')))
# model.load_state_dict(torch.load("/media/hero/Study/User/Study/data/model_out/model_/model_upscale_srcnn.pth"))
model.eval()

cpu


SRCNN(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU(inplace=True)
  (conv3): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
)

In [60]:
import os
# scale = 10
video_name = 'outputs/video.avi'
path_test = "/media/hero/Study/User/Project/image_processing/image_general/processed/"
paths = os.listdir(path_test)
# print(paths)
upscale = 2
frame = cv2.imread(path_test + paths[0])
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))


for idx, path in enumerate(paths):
    path = path_test + path
    # image = Image.open(path).convert('RGB')
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.imread(path)
    image = cv2.resize(image, (image.shape[1]*upscale, image.shape[0]*upscale))
    image = np.array(image, dtype=np.float32)
    image = image/255
    image = image.transpose([2, 0, 1])
    image_processed = torch.tensor([image], dtype=torch.float, device=device)
    outputs = model(image_processed)
    # print(outputs.shape)
    out = np.array(outputs.detach().numpy()[0].T*255, dtype=np.uint8)
    out = out.transpose([1,0,2])
    # idx = random.randrange(1,100)
    print(out.shape)
    test = f"/media/hero/Study/User/Study/data/model_out/images_video/Image_high_quality_{idx}.png"
    # print(torch.from_numpy(np.asarray(outputs.T)))

    cv2.imwrite(test, out)
    video.write(out)
# cv2.imshow("image_high_quality", out)
# cv2.waitKey(0)
cv2.destroyAllWindows()
video.release()



(1024, 1024, 3)
(1024, 1024, 3)
(1024, 1024, 3)
(1024, 1024, 3)
(604, 604, 3)
(604, 606, 3)
(606, 604, 3)
(606, 606, 3)
(936, 1200, 3)
